In [ ]:
#pip install pycaret
#pip install pycaret-nightly

In [23]:
import os
import pandas as pd
import numpy as np

In [2]:
directory= 'C:\\Users\\USER\\PycharmProjects\\ERC\\'
os.chdir(directory)

In [12]:
data_n = 'temp' #temp, ibi, eda
method = 'umap' #pca, tsne, lle, umap

In [13]:
data = pd.read_csv('data/red/{}_{}.csv'.format(data_n, method))                  

Emotion 분류
---

In [27]:
from pycaret.classification import *

emo_clf = setup(data, target='Emotion', session_id = 123)
emo_best_model = compare_models(sort='F1')

,Description,Value
0,Session id,1973
1,Target,Emotion
2,Target type,Multiclass
3,Target mapping,"angry: 0, happy: 1, neutral: 2, sad: 3, surprise: 4"
4,Original data shape,"(1000, 6)"
5,Transformed data shape,"(1000, 6)"
6,Transformed train set shape,"(700, 6)"
7,Transformed test set shape,"(300, 6)"
8,Ordinal features,1
9,Numeric features,4


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
catboost,CatBoost Classifier,0.7186,0.9067,0.7186,0.7157,0.7115,0.6482,0.6511,1.0440
lightgbm,Light Gradient Boosting Machine,0.7086,0.8952,0.7086,0.7073,0.7027,0.6357,0.6382,0.3010
rf,Random Forest Classifier,0.7086,0.9126,0.7086,0.7068,0.7006,0.6357,0.6392,0.2600
et,Extra Trees Classifier,0.7086,0.9179,0.7086,0.7013,0.6998,0.6357,0.6382,0.2630
xgboost,Extreme Gradient Boosting,0.7043,0.8935,0.7043,0.7019,0.6990,0.6304,0.6325,0.2460
gbc,Gradient Boosting Classifier,0.6714,0.8951,0.6714,0.6705,0.6662,0.5893,0.5916,0.3450
dt,Decision Tree Classifier,0.6457,0.7786,0.6457,0.6512,0.6423,0.5571,0.5600,0.1970
lda,Linear Discriminant Analysis,0.6500,0.8896,0.6500,0.6379,0.6364,0.5625,0.5662,0.2000
nb,Naive Bayes,0.6429,0.8813,0.6429,0.6346,0.6318,0.5536,0.5568,0.1940
qda,Quadratic Discriminant Analysis,0.6429,0.8841,0.6429,0.6312,0.6287,0.5536,0.5577,0.1970


Processing:   0%|          | 0/71 [00:00<?, ?it/s]

Arousal/Valence 회귀
---

In [18]:
from pycaret.regression import *

arou_clf = setup(data, target='Arousal', , session_id = 123)
arou_best_model = compare_models(n_select=1)

,Description,Value
0,Session id,187
1,Target,Arousal
2,Target type,Regression
3,Original data shape,"(1000, 6)"
4,Transformed data shape,"(1000, 10)"
5,Transformed train set shape,"(700, 10)"
6,Transformed test set shape,"(300, 10)"
7,Ordinal features,1
8,Numeric features,3
9,Categorical features,2


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,0.1875,0.0644,0.2531,0.6099,0.0611,0.0593,0.5280
rf,Random Forest Regressor,0.1829,0.0656,0.2553,0.6033,0.0615,0.0577,0.2290
et,Extra Trees Regressor,0.1682,0.0686,0.2611,0.5894,0.0629,0.0531,0.2230
lightgbm,Light Gradient Boosting Machine,0.1943,0.0685,0.2609,0.5852,0.0628,0.0611,0.2000
gbr,Gradient Boosting Regressor,0.2040,0.0699,0.2637,0.5756,0.0636,0.0646,0.1920
xgboost,Extreme Gradient Boosting,0.1866,0.0723,0.2678,0.5629,0.0644,0.0587,0.1950
ada,AdaBoost Regressor,0.2215,0.0800,0.2821,0.5204,0.0677,0.0699,0.1900
ridge,Ridge Regression,0.2246,0.0825,0.2857,0.5048,0.0688,0.0713,0.1720
br,Bayesian Ridge,0.2246,0.0826,0.2858,0.5048,0.0688,0.0713,0.1810
lr,Linear Regression,0.2246,0.0826,0.2858,0.5047,0.0688,0.0712,0.1770


Processing:   0%|          | 0/85 [00:00<?, ?it/s]

In [19]:
new_data = data.drop('Arousal', axis=1)
prediction = predict_model(arou_best_model, data=new_data)
prediction

,Emotion,Sex,Valence,0,1,prediction_label
0,neutral,M,3.0,4.856936,-8.995329,3.110966
1,neutral,F,2.9,12.758418,-10.600869,3.068160
2,neutral,M,3.8,18.419182,8.723641,3.194926
3,neutral,F,2.6,12.795440,16.352417,3.373888
4,neutral,F,3.4,11.964708,20.886423,3.534680
...,...,...,...,...,...,...
995,sad,M,2.5,13.642965,9.673536,2.872994
996,sad,M,2.7,-5.824087,-6.545642,2.517513
997,sad,F,2.4,8.380595,6.920435,2.898083
998,sad,M,2.3,12.650342,10.677346,2.739006


In [21]:
#concordance_correlation_coefficient

def ccc(y_pred, y_true):
    """Concordance correlation coefficient."""
    # Remove NaNs
    df = pd.DataFrame({
        'y_true': y_true,
        'y_pred': y_pred
    })
    df = df.dropna()
    y_true = df['y_true']
    y_pred = df['y_pred']
    # Pearson product-moment correlation coefficients
    cor = np.corrcoef(y_true, y_pred)[0][1]
    # Mean
    mean_true = np.mean(y_true)
    mean_pred = np.mean(y_pred)
    # Variance
    var_true = np.var(y_true)
    var_pred = np.var(y_pred)
    # Standard deviation
    sd_true = np.std(y_true)
    sd_pred = np.std(y_pred)
    # Calculate CCC
    numerator = 2 * cor * sd_true * sd_pred
    denominator = var_true + var_pred + (mean_true - mean_pred)**2
    return numerator / denominator

In [26]:
ccc_arou = ccc(prediction['prediction_label'], data['Arousal'])
ccc_arou = round(ccc_arou*100,2)
ccc_arou

88.02